In [1]:
import Cython
%load_ext Cython

In [2]:
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm

# Базовый пример использования Cython
В данном пункте приведем пример простой функции на языке python. В качестве функции рассматривалась функция, которая суммирует числа от $0$ до $N$. В эксперименте сравнивается скорость работы данной функции в следующих случаях: 
* функция реализована на python без предварительной компиляции; 
* функция реализована на pypy;
* функция реализована на python но предварительно скомпилирована;
* функция реализована на cython с типизацией объектов.

## Модели

In [3]:
%%cython

def CythonFunc(N):
    ret = 0
    for n in range(N):
        ret += n
    return ret

def CythonTypedFunc(int N):
    cdef long ret = 0
    cdef long n
    for n in range(N):
        ret += n
    return ret

In [4]:
def Func(N):
    ret = 0
    for n in range(N):
        ret += n
    return ret

## Comparison
Для сравнения рассматривается $N = 10000000$ для всех моделей, а также каждая функция вызывается $200$ раз для усреднения результата.

In [5]:
N = 10000000
number_of_experiment = 200

#### Python Func
Время работы функции для базовой реализации на python.

In [6]:
list_of_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    start = time.time()
    Func(N)
    end = time.time()
    
    list_of_times.append(end-start)
    
    iterable.set_postfix_str('(time={})'.format(np.mean(list_of_times)))
print('(time={})'.format(np.mean(list_of_times)))


(time=0.6191353499889374)


#### Cython Func
Время работы скомпилированной программы на cython без добавления типизации объектов.

In [7]:
list_of_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    start = time.time()
    CythonFunc(N)
    end = time.time()
    
    list_of_times.append(end-start)
    
    iterable.set_postfix_str('(time={})'.format(np.mean(list_of_times)))
    
print('(time={})'.format(np.mean(list_of_times)))


(time=0.4386326432228088)


#### Cython Typed Func
Время работы скомпилированной программы на cython c добавлениям типизации объектов.

In [8]:
list_of_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    start = time.time()
    ret = CythonTypedFunc(N)
    end = time.time()
    
    list_of_times.append(end-start)
    
    iterable.set_postfix_str('(time={})'.format(np.mean(list_of_times)))
print('(time={})'.format(np.mean(list_of_times)))


(time=1.71661376953125e-06)


## Result

В данном простом примере получили следующие оценки времени работы функций:

| Функция  | Время |
| ------------- | ------------- |
| Python Func  | 619 ms  |
| Cython Func  | 439 ms  |
| Cython Typed Func  | 0.172 ms |


# NumPy in Cython

In [9]:
%%cython
import numpy as np
cimport numpy as np
np.import_array()

cpdef return_zeros():
    return np.zeros([10, 15], dtype=np.int32)

cpdef return_empty():
    cdef np.npy_intp dims[2]
    dims[0] = 10
    dims[1] = 15
    cdef np.ndarray data = np.PyArray_SimpleNew(2, dims, np.NPY_INT32)
    np.PyArray_FILLWBYTE(data, 0)
    return data